# Clone precomputed models


gully  
April 26, 2021

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

import seaborn as sns
sns.set_context('paper', font_scale=2)

In [ ]:
import torch
from blase.multiorder import MultiOrder
from blase.datasets import HPFDataset

In [ ]:
import numpy as np
from scipy.signal import find_peaks, find_peaks_cwt, peak_prominences, peak_widths
from scipy.ndimage import gaussian_filter1d

In [ ]:
device = "cpu"
data = HPFDataset("../test/data/Goldilocks_20191022T013208_v1.0_0003.spectra.fits")
model = MultiOrder(device=device, wl_data=data.data_cube[6, :, :])
spectrum = model.forward(5)

In [ ]:
smoothed_flux = gaussian_filter1d(model.flux_native.cpu(), sigma=10.0)

In [ ]:
peaks, _ = find_peaks(-smoothed_flux, distance=10, prominence=0.05)

In [ ]:
prominence_data = peak_prominences(-smoothed_flux, peaks)
width_data = peak_widths(-smoothed_flux, peaks, prominence_data=prominence_data)

In [ ]:
prominences, left, right = prominence_data
widths, width_heights, left_ips, right_ips = width_data

In [ ]:
def lorentzian_line(lam_center, width, amplitude, wavelengths):
    '''Return a Lorentzian line, given properties'''
    return amplitude/3.141592654 * width/(width**2 + (wavelengths - lam_center)**2)

In [ ]:
def gaussian_line(lam_center, width, amplitude, wavelengths):
    '''Return a Gaussian line, given properties'''
    return amplitude/(width*torch.sqrt(torch.tensor(2*3.14159))) * torch.exp(-0.5*((wavelengths - lam_center) / width)**2)

In [ ]:
def black_body(Teff, wavelengths, scale):
    '''Make a black body spectrum given Teff and wavelengths'''
    unnormalized = 1/wavelengths**5 * 1/(torch.exp(1.4387752e-2/(wavelengths*1e-10*Teff)) - 1)
    normalized = unnormalized / torch.mean(unnormalized)
    return scale*normalized

In [ ]:
lam_centers = model.wl_native[peaks]

Convert the FWHM in units of Angstroms: $$\sigma(Angstroms) = FWHM\frac{pixels}{1} \times \frac{Angstrom}{pixel} \times \frac{1}{2.355}$$

In [ ]:
d_lam = np.diff(model.wl_native.cpu())[peaks]
widths_angs = torch.tensor(widths * d_lam / 2.355) * 0.83804203 *0.77116 * 1.58# Experimentally determined

The prominence scale factor may not be exactly 1.

In [ ]:
prominence_scale_factor = 0.461*0.55736 *1.67# Experimentally determined
amplitudes = torch.tensor(prominences * prominence_scale_factor)

Temporarily tilt the cloned model towards the smoothed spectrum and offset for clarity.

## Apply Neural Network Training and Backpropagation

In [ ]:
import torch
from torch import nn

In [ ]:
class PhoenixEmulator(nn.Module):
    r"""
    A PyTorch layer that provides a parameter set and transformations to clone precomputed synthetic spectra.

    """

    def __init__(self):
        super().__init__()

        # Read in the synthetic spectra at native resolution
        #self.wl_native, self.flux_native = self.read_native_PHOENIX_model(4700, 4.5)

        
        self.amplitudes = nn.Parameter(
            torch.tensor(amplitudes, requires_grad=True, dtype=torch.float64)
        )
        self.widths = nn.Parameter(
            torch.tensor(widths_angs, requires_grad=True, dtype=torch.float64)
        )
        self.lam_centers = nn.Parameter(
            torch.tensor(lam_centers, requires_grad=True, dtype=torch.float64)
        )

        self.teff = nn.Parameter(
            torch.tensor(5700, requires_grad=True, dtype=torch.float64)
        )
        
        self.bb_scale = nn.Parameter(
            torch.tensor(1.0, requires_grad=True, dtype=torch.float64)
        )
        

    def forward(self):
        """The forward pass of the spectral model

        Returns:
            (torch.tensor): the 1D generative spectral model destined for backpropagation parameter tuning
        """

        output = lorentzian_line(self.lam_centers.unsqueeze(1), 
                          self.widths.unsqueeze(1), 
                          self.amplitudes.unsqueeze(1), model.wl_native.unsqueeze(0))
        
        net_spectrum = 1-output.sum(0)
        correction_factor = black_body(self.teff, model.wl_native, self.bb_scale)
        final = net_spectrum * correction_factor
        return final

In [ ]:
%%capture
emulator = PhoenixEmulator()

In [ ]:
%%time
cloned_spectrum = emulator.forward()

In [ ]:
from tqdm import trange
import torch.optim as optim
import time

In [ ]:
target = torch.tensor(smoothed_flux)

In [ ]:
loss_fn = nn.MSELoss(reduction="mean")
optimizer = optim.Adam(model.parameters(), 0.02)
n_epochs = 50
losses = []

t0 = time.time()
t_iter = trange(n_epochs, desc="Training", leave=True)
for epoch in t_iter:
    emulator.train()
    yhat = emulator.forward()
    loss = loss_fn(yhat, target)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    losses.append(loss.item())